In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import serial
import pandas as pd
from time import sleep
import numpy as np
from IPython import display
import pandas as pd
from datetime import datetime
import os

In [2]:
class verdi_command:
    def __init__(self, nickname = 'Set Power (W)', command = b'?SP\r\n', return_bytes = 8):
        self.nickname = nickname
        if command[-2:] != b'\r\n':
            self.command = command + b'\r\n'
        else:
            self.command = command
        self.return_bytes = return_bytes
    def send_command(self, ser):
        """ writes verdi command to serial port ser, returns response from laser"""
        ser.write(self.command)
        #while ser.inWaiting() < self.return_bytes:
        #    pass
        #response = ser.read(self.return_bytes)
        response = ser.readline()
        if response[-2:] != b'\r\n':
            print(ser.inWaiting())
            response += ser.read(ser.inWaiting())
            return response
        else:
            return response

commands = [
    verdi_command('Set Power (W)', b'?SP', 8),
    verdi_command('Output Power (W)', b'?P', 7),
    verdi_command('Current (A)', b'?C', 6),
    verdi_command('Current Delta (A)', b'?CD', 5),
    verdi_command('Baseplate Temp (C)', b'?BT', 7),
    verdi_command('Diode 1 Current (A)', b'?d1c', 6),
    verdi_command('Diode 1 Photocell Voltage (V)', b'?d1pc', 6),
    verdi_command('Diode 1 Set Temp (C)', b'?d1st', 7),
    verdi_command('Diode 1 Temp (C)', b'?d1t', 7),
    verdi_command('Diode 1 Heatsink Temp (C)', b'?d1hst', 7),
    verdi_command('Diode 1 Hours (hr)', b'?d1h', 10),
    verdi_command('Diode 1 RCF', b'?d1rcf', 6),
    verdi_command('Diode 1 Max Current (A)', b'?d1rcm', 7),
    verdi_command('Diode 2 Current (A)', b'?d2c', 6),
    verdi_command('Diode 2 Photocell Voltage (V)', b'?d2pc', 6),
    verdi_command('Diode 2 Set Temp (C)', b'?d2st', 7),
    verdi_command('Diode 2 Temp (C)', b'?d2t', 7),
    verdi_command('Diode 2 Heatsink Temp (C)', b'?d2hst', 7),
    verdi_command('Diode 2 Hours (hr)', b'?d2h', 10),
    verdi_command('Diode 2 RCF', b'?d2rcf', 6),
    verdi_command('Diode 2 Max Current (A)', b'?d2rcm', 7),
    verdi_command('Etalon Temp (C)', b'?et', 7),
    verdi_command('Vanadate Set Temp (C)', b'?vst', 7),
    verdi_command('Vanadate Temp (C)', b'?vt', 7),
    verdi_command('Power Supply Hours (hr)', b'?psh', 10),
    verdi_command('Laser Head Hours (hr)', b'?HH', 10),
    verdi_command('Vanadate Temp Servo Drive', b'?VD', 10),
    verdi_command('Diode 1 Temp Servo Drive', b'?d1td', 10),
    verdi_command('Diode 2 Temp Servo Drive', b'?d2td', 10),
    verdi_command('Etalon Temp Servo Drive' , b'?ed', 10),
]

#### Define the serial interface ####
verdi_port = 'COM4' #verdi com port
verdi_baud = 19200  #verdi baud rate, default is 19200
verdi_stop = serial.STOPBITS_ONE
verdi_parity = serial.PARITY_NONE 
verdi_flow = False
#### Open Serial Connection ####
try:
    ser
except:
    ser = serial.Serial(port = verdi_port,
                        baudrate = verdi_baud, 
                        stopbits = verdi_stop,
                        parity = verdi_parity,
                        rtscts = verdi_flow)

In [ ]:
bytearray(5)

In [ ]:
ser.isOpen()
from time import sleep
print(bytearray(5))
while True:
    ser.write(bytearray(5)+b'\r\n')
    #ser.write(b'Hello world')
    sleep(.5)
    #print(str(ser.readline()))

In [ ]:
def write_verdi_header(datafile, commands):
    f = open(datafile, 'w')
    #f.write('Time\t')
    for c in commands:
        if c != commands[-1]:
            f.write(c.nickname+'\t')
        else:
            f.write(c.nickname+'\n')
    f.close()

def get_verdi_data(datafile, ser, commands, mode='a'):
    """Records the responses for each command in 'commands' from the verdi on Serial port 'ser' into the file 'datafile'"""
    s = ''
    
    # flush the input buffer if there are bits waiting
    if ser.inWaiting() > 0:
        ser.read(ser.inWaiting())
    
    # add the date and time for the current data point
    s += str(datetime.now())+'\t'
    
    # loop over the commands and get their responses
    for c in commands[:-1]:
        r = float(c.send_command(ser))
        s += '%.8e\t'%r
    r = float(commands[-1].send_command(ser))
    s += '%.8e\n'%r
    
    # write to the datafile
    f = open(datafile, mode)
    f.write(s)
    f.close()

datafile = 'dat.txt'
write_verdi_header(datafile, commands)
maxiter = 1000
d = datetime.now()
for i in range(maxiter):
    if i%10 == 0:
        print(datetime.now()-d, i)
    get_verdi_data(datafile, ser, commands, mode='a')
            

In [ ]:
f = open(datafile,'r')
f.read()
f.close()
df = pd.read_csv(datafile, sep='\t', index_col=0, parse_dates=True)
df.plot(y=['Diode 1 Temp (C)','Diode 2 Temp (C)','Baseplate Temp (C)'])
ser.close()
#df.plot(y='Etalon Temp (C)')

In [ ]:
pd.read_csv(datafile, sep='\t', index_col=0)

In [ ]:
maxiter = 1800

data = np.zeros((maxiter, len(commands)+1))
header = ['Time']
for vc in commands:
    header.append(vc.nickname)
df = pd.DataFrame(data, columns = header)        
df.loc[0,'Time']

In [ ]:
for i in range(maxiter):
    df.iloc[i,0] = datetime.now()
    j = 1
    for vc in commands:
        df.iloc[i,j] = float(vc.send_command(ser))
        j += 1

In [ ]:
df.to_csv('Z:\\file.csv')

In [ ]:
plt.figure()
df.plot(x='Time',y=['Diode 1 Temp (C)', 'Diode 2 Temp (C)', 'Baseplate Temp (C)','Vanadate Temp (C)'])
plt.legend(loc=2, bbox_to_anchor=(1.05,1))
plt.ylabel('Temperature (C)')

In [ ]:
vc = verdi_command('Power Supply Hours (hr)', b'?psh', 10)
float(vc.send_command(ser))

In [ ]:
%%timeit
ser.write(b'?P\r\n')
waiting = ser.inWaiting()
while ser.inWaiting() != 7:
    pass
message = ser.read(ser.inWaiting())
num = float(message)

In [ ]:
#### Record Data ####
for i in range(10):
    ser.write(b'?P\r\n')
    waiting = ser.inWaiting()
    while ser.inWaiting() != 7:
        pass
    message = ser.read(ser.inWaiting())
    num = float(message)
    print(message,num)

ser.close()

In [ ]:
ser.close()

In [ ]:
ax = plt.subplot(111)

x = np.zeros(10)
y = np.zeros(10)
for i in range(len(x)):
    x[i] = i
    y[i] = i
    display.clear_output(wait=True)
    display.display(plt.gcf())
    ax.cla()
    ax.set_ylim(0,10)
    ax.set_xlim(0,10)
    ax.plot(x,y)
    